<a href="https://colab.research.google.com/github/vashic/Object-Detection/blob/master/chip_finalfiles.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install rasterio
import rasterio as rst
import numpy as np
!pip install geojson
import geojson
from imageio import imwrite
import glob

     |████████████████████████████████| 18.2MB 1.4MB/s 


In [ ]:
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)
!ln -s /content/gdrive/My\ Drive/FinalForUpload /content/crater
!ls /content/crater

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive
B01_010018_1993_XN_19N191W.tiff  F01_036008_2007_XN_20N189W.tiff
Craters.geojson			 G17_024826_2003_XN_20N188W.tiff
Craters_new.geojson		 P12_005733_1985_XI_18N190W.jp2
Craters_old.geojson		 P13_006234_1982_XN_18N190W.tiff
D01_027648_2027_XN_22N189W.jp2	 P14_006524_1968_XN_16N188W.tiff
D20_035019_1986_XN_18N189W.tiff  P20_008739_2013_XN_21N191W.tiff
data				 P22_009807_1987_XN_18N190W.tiff
data_trial


In [ ]:
image_folder = '/content/crater'

%cd crater/
#Extracting all tiff images
im_file_list_tiff=glob.glob('*.tiff')
print(len(im_file_list_tiff))
#Extracting a list of all jp2 images
im_file_list_jp2=glob.glob('*.jp2')
print(len(im_file_list_jp2))
#Creating a list of all image files
im_file_list=im_file_list_tiff+im_file_list_jp2
print(len(im_file_list))
print(im_file_list[0])
%cd ..


/content/gdrive/.shortcut-targets-by-id/1ul9UlFeLVh6_RlgjU1hamnY0kIhLxPMe/FinalForUpload
8
2
10
B01_010018_1993_XN_19N191W.tiff
/content/gdrive/.shortcut-targets-by-id/1ul9UlFeLVh6_RlgjU1hamnY0kIhLxPMe


In [ ]:
# saves chips of size 512x512
# along with labels in YOLO format
def save_files(image, info):
    
    for k in info.keys():
        #print('\n',k)
        y, x = info[k]['center'][0], info[k]['center'][1] #Because of the way the centers are stored we have y coordinate first and then x
        x=int(x)
        y=int(y)
        if y<=256:
          print('y<256')
          continue
        elif x<=256:
          print('x<256')
          continue

        # saving image chip
        iname = '/content/crater/to_augment/{}_{}.jpg'.format(im_file, k)
        #imwrite(iname, image[max((x-256), 0):min((x+256),image.shape[0]),max(0,(y-256)):min((y+256),image.shape[1])])
        im_trial=image[y-256:y+256,x-256:x+256]
        if ((im_trial.shape[0]==512) & (im_trial.shape[1]==512)):
          imwrite(iname, im_trial)
        else:
          print(info[k]['center'][1])
          continue

        # saving label
        file = open(iname.replace('.jpg', '.txt'), 'a')
        for point in info[k]['bbox']:
            [[y_bot, x_bot], [y_top, x_top]] = point #X and Y swap as before

            xc = (x_bot+x_top)//2 - x + 256
            yc = (y_bot+y_top)//2 - y + 256
            w = abs(x_bot-x_top)
            h = abs(y_bot-y_top)

            # 0 means first object i.e. crater
            lab = '0 {} {} {} {}\n'.format(xc/512, yc/512, w/512, h/512)
            file.write(lab)
        file.close()

In [ ]:
def extract_im(imfile):
  # Replace the number is the bracket from 0 to len(im_file_list)-1 to work for the image of your choice.
  dataset=rst.open(image_folder+'/'+im_file)
  # RasterYSize to get the number of rows, RasterXSize to get the number of columns and RasterCount to get bands
  num_rows,num_cols = dataset.shape
  num_bands = dataset.count
  crs=dataset.crs
  transform=dataset.transform
  metadata=dataset.meta
  description=dataset.descriptions
  data_as_arr=dataset.read(1)
  fileg='/content/crater/Craters_new.geojson'
  # reading GEOJSON files
  with open(fileg, 'r') as f:
    labelg = geojson.load(f)
  #Loop through the features of the image collection to convert x y to rows and columns.
  for x in range(0,len(labelg['features'])):
    labelg['features'][x]['properties']['centroid_row'],labelg['features'][x]['properties']['centroid_col']=rst.transform.rowcol(transform,labelg['features'][x]['properties']['centroid_x'],labelg['features'][x]['properties']['centroid_y'])
    labelg['features'][x]['properties']['min_row'], labelg['features'][x]['properties']['min_col']=rst.transform.rowcol(transform,labelg['features'][x]['properties']['min_x'], labelg['features'][x]['properties']['min_y'])
    labelg['features'][x]['properties']['max_row'], labelg['features'][x]['properties']['max_col']=rst.transform.rowcol(transform,labelg['features'][x]['properties']['max_x'], labelg['features'][x]['properties']['max_y'])
  lab=labelg['features'][:]
  #Extract only those features overlying the image we are querying
  features=[]
  for x in range(0,len(lab)):
    if lab[x]['properties']['im_path']==im_file:
      features.append(lab[x])
  feature_collection=geojson.FeatureCollection(features)
  # extracting coordinates of bounding boxes
  coords, center_list = [], []
  for p in range(0,len(feature_collection['features'])):
    x_bot=feature_collection['features'][p]['properties']['min_row']
    y_bot=feature_collection['features'][p]['properties']['min_col']
    x_top=feature_collection['features'][p]['properties']['max_row']
    y_top=feature_collection['features'][p]['properties']['max_col']
    #Stores the bb centre coordinates
    center_list.append([feature_collection['features'][p]['properties']['centroid_row'], feature_collection['features'][p]['properties']['centroid_col']])
    #Stores the bb bottom left and top right
    q=[[x_bot,y_bot],[x_top,y_top]]
    coords.append(q)
  #coords = np.array(coords)
  coords = np.array(coords)
  from sklearn.cluster import DBSCAN
  eps = 220
  dbscan = DBSCAN(min_samples=1, eps=eps)
  x = np.array(center_list)
  y = dbscan.fit_predict(x)
  # storing centroid of clusters
  info = {}
  for i in range(y.max()+1):
    # calculates the max and min coords of all the
    # bounding boxes present in the cluster
    mi_x, mi_y = x[np.where(y==i)[0]].min(axis=0)
    ma_x, ma_y = x[np.where(y==i)[0]].max(axis=0)

    item = {}
    item['center'] = [(mi_x+ma_x)//2, (mi_y+ma_y)//2]
    item['bbox'] = coords[np.where(y==i)[0]].tolist()

    info[i] = item
  save_files(data_as_arr,info)

In [ ]:
for i in im_file_list:
  im_file=i
  extract_im(im_file)
  print(i,' of ',len(im_file_list), '\n')

B01_010018_1993_XN_19N191W.tiff  of  10 

D20_035019_1986_XN_18N189W.tiff  of  10 

G17_024826_2003_XN_20N188W.tiff  of  10 

P13_006234_1982_XN_18N190W.tiff  of  10 

P14_006524_1968_XN_16N188W.tiff  of  10 

P22_009807_1987_XN_18N190W.tiff  of  10 

P20_008739_2013_XN_21N191W.tiff  of  10 

F01_036008_2007_XN_20N189W.tiff  of  10 

D01_027648_2027_XN_22N189W.jp2  of  10 

P12_005733_1985_XI_18N190W.jp2  of  10 

